# Hands on Example - Simulating Bond

In [14]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
import simulate_multi_cty_bond as smcb
import simulate_bond as sb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define Variables

In [15]:
#define nominals
nominal_sng = 150.0
nominal_mlt = 200.0

#define premium and risk free rate
premium = 0.04
rf_rate = 0.01

#number countries
countries = ['01','02']

#specify if average annual returns or compounded annual returns are wanted
ann_com = True

#specify payouts and damages per event
pay_dam_df_c1 = pd.DataFrame([{'pay': np.array(140.0), 'damage': np.array(150.0), 'year': 0},
                              {'pay': np.array(50.0), 'damage': np.array(50.0), 'year': 5}])

pay_dam_df_c2 = pd.DataFrame([{'pay': np.array(40.0), 'damage': np.array(35.0), 'year': 1},
                              {'pay': np.array(160.0), 'damage': np.array(200.0), 'year': 5}])

pay_dam_df_dic = {'01': pay_dam_df_c1,
                  '02': pay_dam_df_c2}

## Simulate Single-Country Bonds

### Country 01

In [16]:
exp_loss_pool_c1, att_prob_pool_c1, es_metrics_c1 = sb.init_exp_loss_att_prob_simulation(pay_dam_df_c1, nominal_sng)
metrics_pool_c1, returns_pool_c1 = sb.init_bond_simulation(pay_dam_df_c1, premium, rf_rate, nominal_sng, ann_com)

Expected Loss = 0.10555555555555556
Attachment Probability = 0.16666666666666666


In [17]:
es_metrics_c1

{'VaR_99_ann': 0.8673333333333337,
 'VaR_99_tot': 0.9153333333333332,
 'VaR_95_ann': 0.6033333333333329,
 'VaR_95_tot': 0.8433333333333332,
 'ES_99_ann': 0.9333333333333333,
 'ES_99_tot': 0.9333333333333333,
 'ES_95_ann': 0.9333333333333333,
 'ES_95_tot': 0.9333333333333333}

In [18]:
metrics_pool_c1

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments
0,0.04,-0.284152,-0.538187,0.95,-10.0,95.0


### Country 02

In [19]:
exp_loss_pool_c2, att_prob_pool_c2, es_metrics_c2 = sb.init_exp_loss_att_prob_simulation(pay_dam_df_c2, nominal_sng)
metrics_pool_c2, returns_pool_c2 = sb.init_bond_simulation(pay_dam_df_c2, premium, rf_rate, nominal_sng, ann_com)

Expected Loss = 0.12777777777777777
Attachment Probability = 0.25


In [20]:
es_metrics_c2

{'VaR_99_ann': 0.9193333333333338,
 'VaR_99_tot': 0.9779999999999999,
 'VaR_95_ann': 0.5966666666666662,
 'VaR_95_tot': 0.8899999999999997,
 'ES_99_ann': 1.0,
 'ES_99_tot': 1.0,
 'ES_95_ann': 1.0,
 'ES_95_tot': 1.0}

In [21]:
metrics_pool_c2

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments
0,0.04,-0.332318,-0.753999,0.851852,-40.0,76.666667


## Simulate Multi-Country Bond

In [22]:
exp_loss_pool, att_prob_pool, es_metrics, MES_cty = smcb.init_exp_loss_att_prob_simulation(pay_dam_df_dic, nominal_mlt)
metrics_pool, returns_pool, tot_coverage_pool_cty = smcb.init_bond_simulation(pay_dam_df_dic, premium, rf_rate, nominal_mlt, countries, ann_com)

Expected Loss = 0.17499999999999996
Attachment Probability = 0.3333333333333333


In [23]:
es_metrics

{'VaR_99_ann': 0.9670000000000002,
 'VaR_99_tot': 0.997,
 'VaR_95_ann': 0.8349999999999997,
 'VaR_95_tot': 0.985,
 'ES_99_ann': 1.0,
 'ES_99_tot': 1.0,
 'ES_95_ann': 1.0,
 'ES_95_tot': 1.0}

In [24]:
MES_cty

{0: {'95': 0.25, '99': 0.25}, 1: {'95': 0.75, '99': 0.75}}

In [25]:
metrics_pool

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments
0,0.04,-0.460943,-0.973964,0.893617,-50.0,140.0
